# Cross Population Heritability Writeup

$$
\newcommand{\Bin}{\operatorname{Bin}}
\newcommand{\Var}{\operatorname{Var}}
\newcommand{\Cov}{\operatorname{Cov}}
\newcommand{\Cor}{\operatorname{Cor}}
\newcommand{\E}{\operatorname{E}}
\newcommand{\diag}{\operatorname{diag}}
\newcommand{\tr}{\operatorname{tr}}
$$

# Introduction

This project studies heritability for one phenotype across multiple populations. The quantity we are measuring will depend upon a few factors:

1. Some underlying true cross population heritability
1. LD patterns in the two population
1. Which SNPs are tagged
1. Common- or rare-variant disease architecture

## Questions

1. Are causal effect sizes the same across populations? (Is $\rho\left(b_1,b_2\right) = 1$?)
2. How well can we do cross-population prediction? What can we say about the correlation of joint-fit effect sizes across populations? (Estimate $\rho\left(g_1,g_2\right)$ for practical uses)

## Definitions

| Raw | Normalized | Definition |
|---|---|---|
| $b$ | $\beta$ | Causal effect size |
| $a$ | $\alpha$ | Marginal effect size |
| $g$ | $\gamma$ | Joint effect size |
| $G$ or $X$ | $W$ | Genotypes |
| $S$ | $\Sigma$ | LD Matrix (covariance vs correlation) |
| | | $Y$ - Phenotypes |

### Genotypes

It makes sense to start things off with genotypes and then proceed to effect sizes. We begin with raw genotypes $G_{ij}$ for SNP $i$ (out of $M$) and individual $j$ (out of $N$). $G_{ij} \in \left \{ 0,1,2 \right\}$ and for simplicity, assume that SNPs are in Hardy-Weinberg equilibrium (i.e. $G_{ij} ~ \Bin \left( 2, p_i \right)$ and $\Var \left( G_{ij} \right) = 2 p_i \left( 1-p_i \right)$, where $p_i$ is the allele frequncy of SNP $i$).

$X_{ij}$ are the mean centered genotypes, $X_{ij} = G_{ij} - 2 p_i$. For the most part, this is what I will call the "raw genotypes," although in principle $G_{ij}$ are really the raw genotypes.

$W_{ij}$ are the mean centered and scaled genotypes, $W_{ij} = \frac{G_{ij} - 2 p_i } { \sqrt{ 2 p_i \left( 1-p_i \right)} }$. $W_{ij}$ has the properties that $\E \left ( W_{ij} \right ) = 0$ and $\Var \left ( W_{ij} \right ) = 1$, which is handy. While this writeup will avoid using $W_{ij}$, thinking of things in normalized genotype terms is helpful.

### LD Matrices

The traditional LD matrix is the correlation of genotypes $\Sigma = \Cor \left( G \right) = \E \left( W W^T \right)$ which can be estimated with $\hat{\Sigma} = WW^T / N$. This can also be though of as:

$$\Sigma = \lim_{N\rightarrow\infty} \hat{\Sigma} = \lim_{N\rightarrow\infty} \frac{WW^T}{N} $$

Note that this thinking with limits of infinite sample size is something I will do repeatedly. For this project, I will be more focussed on the covariance of genotypes, $S = \Cov \left( G \right) = \E \left( X X^T \right)$:

$$
\begin{align}
\hat{S} & = \frac{XX^T}{N} \\
S & = \lim_{N\rightarrow\infty} \hat{S} = \lim_{N\rightarrow\infty} \frac{XX^T}{N} \\
\end{align}
$$

Going forward, I will often refer to multiple LD matrices. These will be done with a superscript, $S^{\left(1\right)}$ and $S^{\left(2\right)}$. Also, I will break SNPs and LD matrices into typed and untyped SNPs.

$$
\begin{align}
X & = \left( \matrix{ X_U \\ X_T } \right) \\
S & = \lim_{N\rightarrow\infty} \frac{ X X^T }{N} = \lim_{N\rightarrow\infty} \frac{ \left( \matrix{ X_U \\ X_T } \right) \left( \matrix{ X_U & X_T } \right) }{N} \\
  & = \left( \matrix{ S_{UU} & S_{UT} \\ S_{TU} & S_{TT} } \right) \\
  & = \left( \matrix{ S_{UA} \\ S_{TA} } \right) = \left( \matrix{ S_{AU} & S_{AT} } \right)
\end{align}
$$

### Effect sizes

In hand-wavy terms, here are the effect sizes. Under a model with additive effect sizes, $b_i$ represents the true underlying biological effect, $a_i$ is the marginal effect if you were to fit just this one SNP in infinite population size and $g_i$ is the effect you would get under a joint fit with a subset of SNPs in infinite population size. If you were to fit all the SNPs in a magical infinite population where you basically have observed every possible variant (e.g. no fixed SNPs), then $b = g$.

This is where things get a little hairy with the definitions. Let's start with basic expections:

$$
\begin{align}
\E \left( Y_j | X_{ij} \right) & = X_{ij} a_i \\
\E \left( Y_j | X_{Tj} \right) & = X_{Tj}^T g \\
\E \left( Y_j | X_j \right) & = X_j^T b \\
\end{align}
$$

The reason I say "hairy" is that the individual $b_i$s don't exist in a vacuum. You need to have all the SNPs to go from $b$ to $Y$.

# Causal and marginal effect sizes ($b$ and $a$)

Consider two SNPs, $X_1$ and $X_2$ where $X_1$ is a untyped causal SNP ($b_1 \ne 0$) and $X_2$ is a typed noncausal SNP ($b_2 = 0$) that tags $X_1$ ($S_{12} = \Cov \left( X_1, X_2 \right) \ne 0$). We get the following:

$$
\begin{align}
\E \left( Y_j | X_j \right) & = X_j^T b = \sum_{i=1}^2 X_{ij} b_i = X_{1j} b_1 \\
\E \left( Y_j | X_{2j} \right) & = X_{2j} a \\
X_{2j} \E \left( Y_j | X_{2j} \right) & = X_{2j}^2 a \\
\E \left[ X_{2j} \E \left( Y_j | X_{2j} \right) \right] & = \E \left[ X_{2j}^2 a \right] = S_{22} a \\
a & = S_{22}^{-1} \E \left\{ X_{2j} \E \left[ \E \left( Y_j | X_{1j} \right) | X_2j \right] \right \} \\
  & = S_{22}^{-1} \E \left\{ X_{2j} \E \left[ X_{1j} b | X_2j \right] \right \}
    = S_{22}^{-1} \E \left\{ X_{1j} X_{2j} \right \} b\\
  & = \frac{ S_{12} } { S_{22} } b
\end{align}
$$

Alternative derivation:

$$
\begin{align}
Y & = X_1^T b + \epsilon_b \\
  & = X_2^T a + \epsilon_a \\
a & = \lim_{N\rightarrow\infty} \hat{a}
    = \lim_{N\rightarrow\infty} \frac{ X_2 Y  }{ X_2 X_2^T }
    = \lim_{N\rightarrow\infty} \frac{ X_2 X_1^T }{ X_2 X_2^T } b_1 \\
  & = \frac{ S_{12} } { S_{22} } b
\end{align}
$$

A derivation with many causal SNPs can proceed like the above except $X$ is substituted in place of $X_1$. The result is

$$
\begin{align}
a_i & = \frac{ S_{iA} } { S_{ii} } b \\
a   & = \diag \left( S \right)^{-1} S b
\end{align}
$$

Note here that $\diag \left( S \right)$ is the diagonal elements of $S$ or $\Var \left( G_ij \right)$ which is $2 p_i \left( 1 - p_1 \right)$ under Hardy-Weinberg equilibrium. If $p_i$ is fixed, then $a = \Sigma b$. For simplicity, define $V = \diag \left( S \right)$.

*Bear with me, but I'm testing everything when debugging this project.*

### Test 1 - $a = V^{-1} S b$ for pairwise related SNPs

Construct a matrix

$$ S = \left( \matrix{ V_1 & V_1^{1/2} R V_2^{1/2} \\ V_2^{1/2} R V_1^{1/2} & V_2 } \right) $$

where $R = \diag \left( r_1, r_2, \dots, r_M \right)$ which indicates the correlation between $M$ pairs of SNPs. We will then simulate $X$ from this covariance matrix and test $a = V_2^{-1} V_2^{1/2} R V_1^{1/2} b$. We will also compare this to $a = R b$.

In [1]:
%run library.ipynb
np.set_printoptions(precision=3)

In [2]:
def test_aVSb(R, V1=None, V2=None):
    M = len(R)

    if V1 is None:
        V1 = np.ones(M)

    if V2 is None:
        V2 = np.ones(M)
    
    V12 = np.sqrt(V1 * V2)

    b = nr.normal(size=M)

    ar = b * R
    
    R = np.diag(R * V12)
    
    a = R.dot(b) / V2
    
    V1 = np.diag(V1)
    V2 = np.diag(V2)

    S = np.bmat([[V1, R], [R, V2]])

    X = nr.multivariate_normal([0]*(2*M), S, 10*M).T
    Y = X[:M].T.dot(b)

    ahat = np.linalg.inv(X[M:].dot(X[M:].T)).dot(X[M:].dot(Y))

    return(np.corrcoef(ar, ahat)[0,1], np.corrcoef(a, ahat)[0,1])

#### Test 1a  - $R = rI$, $V_k = I$, $a = rb$

In [48]:
for r in (np.arange(0.1, 1, 0.1)):
    print(r, test_aVSb([r]*100)[1])

0.1 0.290886217951
0.2 0.457356877256
0.3 0.721813744468
0.4 0.782421545535
0.5 0.810341077488
0.6 0.927137003324
0.7 0.95358362679
0.8 0.967292103507
0.9 0.9901406798


As we see, $a$ and $\hat{a}$ deviate as the correlation gets smaller, which is expected.

#### Test 1b - $R \sim 2 \times Beta\left(2,2\right) - 1$, $V_k = I$, $a = Rb$

In [49]:
print(test_aVSb(2*nr.beta(2,2,size=1000)-1)[1])

0.827230793371


#### Test 1c - $R \sim 2 \times Beta\left(2,2\right) - 1$, $V_k = \diag \left( 2 p_ki \left( 1 - p_ki \right) \right)$

In [44]:
M  = 1000
R  = 2*nr.beta(2,2,M)-1
P1 = nr.uniform(0.05, 0.95, M)
P2 = nr.uniform(0.05, 0.95, M)
V1 = 2 * P1 * (1-P1)
V2 = 2 * P2 * (1-P2)
test_aVSb(R, V1, V2)

(0.78692853682744091, 0.83058048563132636)

### Test 2 - Banded LD structure

Now consider a banded matrix and let's try again. Here, $ \Sigma_{ij} = \cases{ \rho^{ \left|i-j\right| } & i - j <= K \\ 0 & \text{otherwise} } $.

In [50]:
rho = 0.4
M = 100
K = 5

Sigma = banded_matrix(rho, M, K)

P   = nr.uniform(0.05, 0.95, M)
V   = 2 * P * (1-P)
V12 = np.sqrt(V)

S = Sigma.toarray() * V12[None,:] * V12[:,None]

b = nr.normal(size=M)

X = nr.multivariate_normal([0]*M, S, 20*M)
Y = X.dot(b)

ahat = X.T.dot(Y) / (X**2).sum(axis=0)

ap = Sigma.dot(b)

a = S.dot(b) / V

print(np.corrcoef(ahat, ap)[0,1], np.corrcoef(ahat, a)[0,1])

0.956866316325 0.983432874518


Everything looks good!

## Causal and marginal correlations

Let's revisit the example with pairs of SNPs, one causal and untyped and one tagged SNP in LD with the causal SNPs. The only difference is now there are two populations. $b_k$ and $a_k$ refers to the effect size in population $k$ and $S_{ij}^{\left(k\right)}$ refers to the covariance between the two SNPs in population k. Now, let's make it a bit more whacky where $r_k = S_{12}^{\left(k\right)} / S_{22}^{\left(k\right)}$ is also a random variable (note that if the allele frequencies for these two SNPs is the same, then $r_k=\Sigma_{12}^{\left(k\right)}$).

$$
\begin{align}
\rho_b & = \Cor \left(b_1, b_2\right) \\
\rho_r & = \Cor \left(r_1, r_2\right) \\
\rho_a & = \Cor \left(a_1, a_2\right) = \Cor \left( r_1 b_1, r_2 b_2 \right) \\
    & = \frac
      { \Cov \left( r_1 b_1, r_2 b_2 \right) }
      { \sqrt{ \Var \left( r_1 b_1 \right) \Var \left( r_2 b_2 \right) } } \\
\Cov \left( r_1 b_1 , r_2 b_2 \right)
  & = \E \left\{ \prod_k \left[ r_k b_k - E \left( r_k b_k \right) \right] \right\} \\
  & = \E \left( r_1 r_2 b_1 b_2 \right) && E \left( r_k b_k \right) = 0 \\
  & = \E \left( r_1 r_2 \right) \E \left( b_1 b_2 \right) && r_k \perp b_l \\
\Var \left( r_k b_k \right)
  & = \E \left\{ \left[ r_k b_k - \E \left( r_k b_k \right) \right]^2 \right\} \\
  & = \E \left[ \left( r_k b_k \right)^2 \right] && E \left( r_k b_k \right) = 0 \\
  & = \E \left( r_k^2 \right) \E \left( b_k^2 \right) && r_k \perp b_k \\
\rho_a
  & = \frac
    { \E \left( r_1 r_2 \right) \E \left( b_1 b_2 \right) }
    { \sqrt{ \E \left( r_1^2 \right) \E \left( b_1^2 \right) \E \left( r_2^2 \right) \E \left( b_2^2 \right) } } \\
  & = \frac
    { \E \left( r_1 r_2 \right) }
    { \sqrt{ \E \left( r_1^2 \right) \E \left( r_2^2 \right) } }
    \frac
    { \E \left( b_1 b_2 \right) }
    { \sqrt{ \E \left( b_1^2 \right) \E \left( b_2^2 \right) } } \\
  & = \rho_r \rho_b && \E \left( r_k \right) = \E \left( b_k \right) = 0
\end{align}
$$

#### Test $\rho_a = \rho_r \rho_b$

1. Simulate $b_1$ and $b_2$ from
$\operatorname{MVN} \left[
  \left( \matrix{ 0 \\ 0 } \right),
  \left( \matrix{ 1 & \rho_b \\ \rho_b & 1 } \right)
\right]$

1. Simulate $r_1^\prime$ and $r_2^\prime$ from a similar MVN. Transform to $r_1$ and $r_2$ which follow $2 * \operatorname{Beta} \left( 2, 2 \right) - 1$. The resulting numbers should have a similar covariance structure.
1. Simulate $X_1$ and $X_2$ from with $S^{\left(1\right)}$ and $S^{\left(2\right)}$ covariance matrices.
1. Calculate $a_k$ and $\hat{a}_k$.
1. Calculate $\rho_a$,  $\Cor\left(a_1,a_2\right)$ and $\Cor\left(\hat{a}_1,\hat{a}_2\right)$

In [55]:
def test_rhoarb (rho_b, rho_r, M=1000):
    b = multivariate_normal_sample(np.zeros(2), equicorrelation_matrix(rho_b, 2), M)
    r = multivariate_normal_sample(np.zeros(2), equicorrelation_matrix(rho_r, 2), M)
    r = 2 * ss.beta(2, 2).isf(ss.norm.sf(r)) - 1

    a    = b * r
    
    I = np.identity(M)

    ahat = list()

    for k in range(2):
        R = np.diag(r[k])
        S = np.bmat([[I, R], [R, I]])
        X = nr.multivariate_normal([0]*2*M, S, 10*M).T
        Y = X[:M].T.dot(b[k])
        ahat.append(X[M:].dot(Y) / (X[M:]**2).sum(axis=1))

    return(rho_b*rho_r, np.corrcoef(b)[0,1]*np.corrcoef(r)[0,1], np.corrcoef(a)[0,1], np.corrcoef(ahat)[0,1])

In [58]:
for i in range(10):
    print("%.3f %.3f %.3f %.3f" % test_rhoarb(0.8, 0.8, 1000))

0.640 0.632 0.636 0.417
0.640 0.634 0.602 0.370
0.640 0.637 0.663 0.443
0.640 0.632 0.631 0.409
0.640 0.633 0.642 0.427
0.640 0.640 0.648 0.458
0.640 0.634 0.659 0.414
0.640 0.633 0.587 0.361
0.640 0.639 0.620 0.420
0.640 0.637 0.597 0.398


The first number is the target $\rho_a$, the second is what it is with the sample correlation of $r$ and $b$, the third is $\Cor\left(a_1, a_2\right)$ and the fourth is $\Cor\left(\hat{a}_1,\hat{a}_2\right)$. The first three numbers are as expected, the fourth is likely reduced due to random noise.

### More complex LD

Let's go back to $a = V_T^{-1} S_{TA} b$. How can we extend this to $\rho_a$? Additionally, what does $\rho_a$ mean in this context? From the form of it, even if the entries $b_ki$ are IID, $a_ki$ are not. So while $\rho_b$ can be expressed in terms of the correlation between two random variables (i.e. $\Cor \left( b_1, b_2 \right) = \rho_b I$), the correlation between $a_1$ and $a_2$ will not be quite so clean. However, we might get around this by thinking in aggregate and with traces. To simplify the notation, let $R_k := \left( V_T^{\left(k\right)} \right)^{-1} S_{TA}^{\left(k\right)}$ and $a_k = R_k b_k$.

$$
\begin{align}
\rho_b & = \Cor \left( b_1i, b_2i \right) \\
\Cor \left( b_1, b_2 \right) & = \rho_b I \\
\tr \left[ \Cor \left( b_1, b_2 \right) \right] & = M \rho_b \\
\Var \left( b_k \right) & = x I & x\text{ is analogous to } h^2 \\
\Cov \left( b_1, b_2 \right) & = \E \left( b_1 b_2^T \right) = x \rho_b I \\
\rho_b
  & = \frac{
    \E \left( b_1^T b_2 \right)
  }{ \sqrt{ 
    \E \left( b_1^T b_1 \right) \E \left( b_2^T b_2 \right)
  }} \\
  & = \frac{ M x \rho_b }{ M x } \\
\rho_a
  & = \frac{
    \E \left( a_1^T a_2 \right)
  }{ \sqrt{ 
    \E \left( a_1^T a_1 \right) \E \left( a_2^T a_2 \right)
  }} \\
  & = \frac{
    \E \left[ b_1^T R_1^T R_2 b_2 \right]
  }{ \sqrt{ \prod_{k\in \left\{1,2\right\}} \E \left( b_k^T R_k^T R_k b_k\right)
  }} \\
\E \left[ b_{k_1}^T R_{k_1}^T R_{k_2} b_{k_2} \right]
  & = \E \left\{ \tr \left[ b_{k_1}^T R_{k_1}^T R_{k_2} b_{k_2} \right] \right\} \\
  & = \E \left\{ \tr \left[ b_{k_2} b_{k_1}^T R_{k_1}^T R_{k_2} \right] \right\} \\
  & = \tr \left\{ \E \left[ b_{k_2} b_{k_1}^T \right] R_{k_1}^T R_{k_2} \right\} \\
  & = \tr \left\{ x \rho_{b_{k_1 k_2}} I R_{k_1}^T R_{k_2} \right\} \\
  & = x \rho_{b_{k_1 k_2}} \tr \left[ R_{k_1}^T R_{k_2} \right] \\
\rho_a & = \rho_b \frac{
  \tr \left[ R_1^T R_2 \right]
}{ \sqrt{
\tr \left[ R_1^T R_1 \right] \tr \left[ R_2^T R_2 \right]
}}
\end{align}
$$

This is a similar form to $\rho_a = \rho_r \rho_b$.

#### Caveat to the above

Suppose all causal SNPs are untyped and not in LD with causal SNPs. The resulting LD matrix will look like:

$$
\begin{align}
S & = \left( \matrix{ S_{UU} & 0 \\ 0 & S_{TT} } \right) \\
R & = \left( \matrix{ R_{UU} & 0 \\ 0 & R_{TT} } \right) \\
\end{align}
$$

The correlation between $a_1$ and $a_2$ will be completely random in this case. However the fraction with the trace will not be zero. The source for this error comes from dividing $\E \left( a_k^T a_k \right)$ because that will be 0 in this case.

#### Testing $\rho_a = \rho_b \frac{
  \tr \left[ R_1^T R_2 \right]
}{ \sqrt{
\tr \left[ R_1^T R_1 \right] \tr \left[ R_2^T R_2 \right]
}}$

Procedure:

1. Sample $b \sim N \left[ \left( \matrix{ 0 \\ 0 } \right), \left( \matrix{ 1 & \rho_b \\ \rho_b & 1 } \right) \right]$
1. Let $\left( \Sigma^{\left(k\right)} \right)^\prime$ be a banded matrix with parameter $\rho_{\Sigma k}$.
1. Let $\Sigma^{\left(k\right)} = \left( \matrix{ \left( \Sigma^{\left(k\right)} \right)^\prime & \rho_{\Sigma k} \left( \Sigma^{\left(k\right)} \right)^\prime \\ \rho_{\Sigma k} \left( \Sigma^{\left(k\right)} \right)^\prime & \left( \Sigma^{\left(k\right)} \right)^\prime } \right)$. The bottom of this matrix are the typed SNPs.
1. Sample $p_{ki} \sim Uniform \left( 0.05, 0.95 \right)$
1. Compute $S^{\left(k\right)} = \left(V^{\left(k\right)}\right)^{\frac{1}{2}} \Sigma^{\left(k\right)} \left(V^{\left(k\right)}\right)^{\frac{1}{2}}$
1. Compute $R_k = \left( V_T^{\left(k\right)} \right)^{-1} S_{TA}^{\left(k\right)}$
1. Compute $a_k = R_k b_k$
1. Sample $X_k \sim N \left(0, S^{\left(k\right)} \right)$
1. Compute $\hat{a}_k$
1. Calculate $\widehat{\Cor}\left(a_1, a_2\right)$, $\widehat{\Cor}\left(\hat{a}_1, \hat{a}_2\right)$ and $\rho_b \frac{
  \tr \left[ R_1^T R_2 \right]
}{ \sqrt{
\tr \left[ R_1^T R_1 \right] \tr \left[ R_2^T R_2 \right]
}}$.

In [5]:
M = 1000
rho_b = 0.8
b = multivariate_normal_sample(np.zeros(2), equicorrelation_matrix(rho_b, 2), 2*M)

Sigma1prime = banded_matrix(0.6, M, 4)
Sigma2prime = banded_matrix(0.2, M, 2)

Sigma1 = np.array(np.bmat([[Sigma1prime, 0.6 * Sigma1prime], [0.6*Sigma1prime, Sigma1prime]]))
Sigma2 = np.array(np.bmat([[Sigma1prime, 0.2 * Sigma1prime], [0.2*Sigma1prime, Sigma1prime]]))

p = nr.uniform(0.05, 0.95, (2,2*M))
v = 2*p*(1-p)
sd = np.sqrt(v)

S1 = Sigma1 * sd[0][:,None] * sd[0][None,:]
S2 = Sigma1 * sd[1][:,None] * sd[1][None,:]

R1 = S1 / v[0][:, None]
R2 = S2 / v[1][:, None]

a = np.array([R1.dot(b[0]), R2.dot(b[1])])

X1 = nr.multivariate_normal([0]*2*M, S1, 10*M)
X2 = nr.multivariate_normal([0]*2*M, S2, 10*M)

ahat1 = X1.T.dot(X1.dot(b[0])) / (X1**2).sum(axis=0)
ahat2 = X2.T.dot(X2.dot(b[0])) / (X2**2).sum(axis=0)

ahat = np.array([ahat1, ahat2])

tr = np.trace(R1[M:].T.dot(R2[M:])) / np.sqrt( np.trace(R1[M:].T.dot(R1[M:])) * np.trace(R2[M:].T.dot(R2[M:])) )

print(np.corrcoef(a[:,M:])[0,1], np.corrcoef(ahat[:,M:])[0,1], rho_b*tr, tr)

0.754153986736 0.878946756593 0.752543052547 0.940678815684


In [4]:
M = 1000
rho_b = 0.8
b = multivariate_normal_sample(np.zeros(2), equicorrelation_matrix(rho_b, 2), 2*M)

Sigma1prime = banded_matrix(0.6, M, 4)
Sigma2prime = banded_matrix(0.2, M, 2)
Zero = np.zeros((M,M))

Sigma1 = np.array(np.bmat([[Sigma1prime, Zero], [Zero, Sigma1prime]]))
Sigma2 = np.array(np.bmat([[Sigma1prime, Zero], [Zero, Sigma1prime]]))

p = nr.uniform(0.05, 0.95, (2,2*M))
v = 2*p*(1-p)
sd = np.sqrt(v)

S1 = Sigma1 * sd[0][:,None] * sd[0][None,:]
S2 = Sigma1 * sd[1][:,None] * sd[1][None,:]

R1 = S1 / v[0][:, None]
R2 = S2 / v[1][:, None]

a = np.array([R1.dot(b[0]), R2.dot(b[1])])

X1 = nr.multivariate_normal([0]*2*M, S1, 10*M)
X2 = nr.multivariate_normal([0]*2*M, S2, 10*M)

ahat1 = X1.T.dot(X1.dot(b[0])) / (X1**2).sum(axis=0)
ahat2 = X2.T.dot(X2.dot(b[0])) / (X2**2).sum(axis=0)

ahat = np.array([ahat1, ahat2])

tr = np.trace(R1[M:].T.dot(R2[M:])) / np.sqrt( np.trace(R1[M:].T.dot(R1[M:])) * np.trace(R2[M:].T.dot(R2[M:])) )

print(np.corrcoef(a[:,M:])[0,1], np.corrcoef(ahat[:,M:])[0,1], rho_b*tr, tr)

0.749695074684 0.858849230562 0.762203881124 0.952754851405


In [1]:
M = 1000
rho_b = 0.8
b = np.concatenate(
    (
        multivariate_normal_sample(np.zeros(2), equicorrelation_matrix(rho_b, 2), M),
        np.zeros((2,M))
    ), 1)

Sigma1prime = banded_matrix(0.6, M, 4)
Sigma2prime = banded_matrix(0.2, M, 2)
Zero = np.zeros((M,M))

Sigma1 = np.array(np.bmat([[Sigma1prime, Zero], [Zero, Sigma1prime]]))
Sigma2 = np.array(np.bmat([[Sigma1prime, Zero], [Zero, Sigma1prime]]))

p = nr.uniform(0.05, 0.95, (2,2*M))
v = 2*p*(1-p)
sd = np.sqrt(v)

S1 = Sigma1 * sd[0][:,None] * sd[0][None,:]
S2 = Sigma2 * sd[1][:,None] * sd[1][None,:]

R1 = S1 / v[0][:, None]
R2 = S2 / v[1][:, None]

a = np.array([R1.dot(b[0]), R2.dot(b[1])])

X1 = nr.multivariate_normal([0]*2*M, S1, 10*M)
X2 = nr.multivariate_normal([0]*2*M, S2, 10*M)

ahat1 = X1.T.dot(X1.dot(b[0])) / (X1**2).sum(axis=0)
ahat2 = X2.T.dot(X2.dot(b[0])) / (X2**2).sum(axis=0)

ahat = np.array([ahat1, ahat2])

tr = np.trace(R1[M:].T.dot(R2[M:])) / np.sqrt( np.trace(R1[M:].T.dot(R1[M:])) * np.trace(R2[M:].T.dot(R2[M:])) )

print(np.corrcoef(a[:,M:])[0,1], np.corrcoef(ahat[:,M:])[0,1], rho_b*tr, tr)

NameError: name 'np' is not defined

In [23]:
M = 5
rho_b = 0.8
b = np.concatenate(
    (
        multivariate_normal_sample(np.zeros(2), equicorrelation_matrix(rho_b, 2), M),
        np.zeros((2,M))
    ), 1)

Sigma1prime = banded_matrix(0.6, M, 4)
Sigma2prime = banded_matrix(0.2, M, 2)
Zero = np.zeros((M,M))

Sigma1 = np.array(np.bmat([[Sigma1prime, Zero], [Zero, Sigma1prime]]))
Sigma2 = np.array(np.bmat([[Sigma1prime, Zero], [Zero, Sigma1prime]]))

p = nr.uniform(0.05, 0.95, (2,2*M))
v = 2*p*(1-p)
sd = np.sqrt(v)

S1 = Sigma1 * sd[0][:,None] * sd[0][None,:]
S2 = Sigma1 * sd[1][:,None] * sd[1][None,:]

R1 = S1 / v[0][:, None]
R2 = S2 / v[1][:, None]

a = np.array([R1.dot(b[0]), R2.dot(b[1])])

X1 = nr.multivariate_normal([0]*2*M, S1, 10*M)
X2 = nr.multivariate_normal([0]*2*M, S2, 10*M)

ahat1 = X1.T.dot(X1.dot(b[0])) / (X1**2).sum(axis=0)
ahat2 = X2.T.dot(X2.dot(b[0])) / (X2**2).sum(axis=0)

ahat = np.array([ahat1, ahat2])

tr = np.trace(R1[M:].T.dot(R2[M:])) / np.sqrt( np.trace(R1[M:].T.dot(R1[M:])) * np.trace(R2[M:].T.dot(R2[M:])) )

print(np.corrcoef(a[:,M:])[0,1], np.corrcoef(ahat[:,M:])[0,1], rho_b*tr, tr)

nan -0.233981172238 0.789857845941 0.987322307426


C:\Users\Kevin\Miniconda3\lib\site-packages\numpy\lib\function_base.py:1957: RuntimeWarning: invalid value encountered in true_divide
  return c / sqrt(multiply.outer(d, d))
